In [1]:
import os
import numpy as np
import cv2
from keras.applications.resnet50 import ResNet50, preprocess_input

# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False)

def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_array = np.expand_dims(img, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

def extract_features_from_folder(image_folder, save_folder):
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    for filename in os.listdir(image_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.gif')):
            img_path = os.path.join(image_folder, filename)
            
            # Preprocess the image and extract features
            img_array = preprocess_image(img_path)
            features = base_model.predict(img_array)
            features = features.flatten()

            # Save the features to a numpy file
            save_path = os.path.join(save_folder, f"{os.path.splitext(filename)[0]}.npy")
            np.save(save_path, features)

if __name__ == "__main__":
    # Set the paths for your image folder and the folder to save features
    image_folder = './images'
    save_folder = './features'

    extract_features_from_folder(image_folder, save_folder)
    print("Feature extraction completed.")


2023-11-13 19:51:18.161562: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-13 19:51:18.245934: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-13 19:51:18.245973: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-13 19:51:18.246011: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-13 19:51:18.257998: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-13 19:51:18.258490: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

1/1 [==============================] - 0s 58ms/step


KeyboardInterrupt: 

In [16]:
#Main code 1
import os
import numpy as np
import cv2
from keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.model_selection import train_test_split

# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False)

def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_array = np.expand_dims(img, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

def extract_features_from_folder(image_folder, save_folder):
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # List all image files in the folder
    image_files = [filename for filename in os.listdir(image_folder) if filename.endswith(('.png', '.jpg', '.jpeg', '.gif'))]

    # Split the image files into train and test sets
    # train_files, test_files = train_test_split(image_files, test_size=0.2, random_state=42)

    # Process the training set
    for filename in image_files:
        img_path = os.path.join(image_folder, filename)
        img_array = preprocess_image(img_path)
        features = base_model.predict(img_array)
        features = features.flatten()
        save_path = os.path.join(save_folder, f"{os.path.splitext(filename)[0]}.npy")
        np.save(save_path, features)

if __name__ == "__main__":
    # Set the paths for your image folder and the folder to save features
    image_folder = './images'
    save_folder = './features2'

    extract_features_from_folder(image_folder, save_folder)
    print("Feature extraction completed.")


1/1 [==============================] - 0s 60ms/step
Feature extraction completed.


In [5]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [13]:
import os


1/1 [==============================] - 1s 860ms/step
Most similar images:
./features2/train_Test_Image_1_460.jpg
./features2/train_Test_Image_1_465.jpg
./features2/train_Test_Image_1_450.jpg
./features2/train_Test_Image_1_475.jpg
./features2/train_Test_Image_1_440.jpg
Query feature extraction completed.


In [ ]:
import numpy as np
import cv2
from keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.metrics.pairwise import cosine_similarity

# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False)


def load_features(feature_folder):
    feature_vectors = []
    image_paths = []

    for filename in os.listdir(feature_folder):
        if filename.startswith("train_") and filename.endswith(".npy"):
            feature_path = os.path.join(feature_folder, filename)
            feature_vector = np.load(feature_path)
            feature_vectors.append(feature_vector)
            image_paths.append(feature_path.replace(".npy", ".jpg"))

    return np.array(feature_vectors), image_paths

def find_most_similar_images(query_features, all_features, image_paths, top_k=5):
    similarities = cosine_similarity(query_features.reshape(1, -1), all_features)
    similar_indices = similarities.argsort()[0][-top_k:][::-1]
    similar_images = [image_paths[i] for i in similar_indices]

    return similar_images

def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_array = np.expand_dims(img, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

def extract_features_from_single_image(query_image_path):
    # Preprocess the query image and extract features
    img_array = preprocess_image(query_image_path)
    features = base_model.predict(img_array)
    features = features.flatten()
    return features

if __name__ == "__main__":
    # Set the paths for your image folder and the folder to save features
    feature_folder = './features2'

    # Load features and image paths
    all_features, image_paths = load_features(feature_folder)
    
    image_folder = './images'
    save_folder = './features2'

    # Example: Extract features from a single image
    query_image_path = './images/Test_Image_1_455.jpg'
    query_features = extract_features_from_single_image(query_image_path)

    similar_images = find_most_similar_images(query_features, all_features, image_paths)

    print("Most similar images:")
    for img_path in similar_images:
        print(img_path)


    print("Query feature extraction completed.")
